# Problem 4 : Estimation - BLP

In [1]:
using Plots, DataFrames, CSV, GLM
using Optim, Distributions, Random, ForwardDiff
using LinearAlgebra,StatsFuns

Random.seed!(111);

In [3]:
df = DataFrame(CSV.File("../data/ps1_ex4.csv"));
x = Array(df[!,[:x, :p]]);

# draw a vector of normal shocks
ν = transpose(rand(MvNormal([0;0], [1 0; 0 1]), 20));

Γ = ones(2,2)
Γ[1,2] = 0

δ = ones(size(df, 1));

# Part 1: BLP

## Inner loop
`get_shares` calculates the shares of each product in a particular market $t$. $\delta$ should be a vector of length $J$; $x$ should be a vector of length $J$; and $v$ should be a vector of length $L$.

`delta_contraction` iterates the $\delta_{jt}$ in a particular market $t$. $\delta$ should be a vector of length $J$; $x$ should be a vector of characteristics with length $J$; $s$ should be a vector of observed shares with length $J$; $v$ should be a vector of length $L$. 

`market_iterate` performs the contraction over each $t$ markets, it recoves $\delta_jt$, which is a vector of length $J \times T$.

In [ ]:
# get shares in a market given some fixed gamma and delta
get_shares = function(δ, Γ, x, v)
    # we want to get share_{jt} using simulated values of $v_i$ (drawn above)
    # shares should be vector of length J
    
    numerator = exp.(δ + x' * Γ * v)
    denominator = sum(numerator) + 1
    shares = sum(numerator ./ denominator) / size(v)[1]  
    
    return shares
end

# inner loop: contraction to find δ
delta_contraction = function(δ₀, Γ, s, x, v, tol)
    
    # here δ is a vector of length J
    δ = δ₀
    err = 1000
    
    while err < tol
        δ_old = δ
        
        # update delta
        δ = δ_old + log.(s) - log.(get_shares(δ_old, Γ, x, v))
        
        # difference 
        err = maximum(abs.(δ - δ_old)) 
    end
    
    return δ
end

# iterate over each market
market_iterate = function(initial, Γ, s_t, x_t, v, tol)
   
    δ = []
    for t in 1:size(s_t)[1]
        s = s_t[t]
        x = x_t[t]
        δ₀ = ones(size(s)[1])
        push!(δ, delta_contraction(δ₀, Γ, s, x, v, tol) ) 
    end
end

## Outer loop

In [ ]:
# 

In [47]:
summed_jt_i = exp.(δ .+ x * Γ * ν') # this returns a 600 (jt) by 20 (i) 
sum(summed_jt_i, dims=2) # this sums across individuals 

600×1 Matrix{Float64}:
     57.00582229965489
    233.16347456628938
     62.23148664334593
    139.0192307176588
      1.9572010631035707e13
      7.5191040125367e9
     64.61729047613422
     55.921108756838976
    176.30447614723312
   4600.2410653501875
      1.1154570965762811e6
      4.802172137882022e8
     54.97046030264811
      ⋮
     92.05096050484761
    183.74617118856838
    707.4496820917077
  87825.27102073307
   3984.880690599852
   6384.160433719341
     54.68272891024609
     57.629424848527094
  89979.25568635199
 175360.58854399397
 112097.32688149536
      3.2869182094897693e19

In [3]:
sum(summed_jt_i, dims=2) # this should sum across goods/markets

LoadError: UndefVarError: summed_jt_i not defined